In [33]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [34]:
from bs4 import BeautifulSoup as bs

In [35]:
import requests

In [36]:
import re

In [37]:
import pandas as pd

In [38]:
url_source = 'https://hh.ru/vacancies/'

In [39]:
specialist = 'python-developer'

In [40]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 YaBrowser/23.7.3.823 Yowser/2.5 Safari/537.36'}
url = url_source + specialist
vac_list = []

In [41]:
def get_information(url, page, headers):
    params = {'page' : page}
    response = requests.get(url, params = params, headers = headers)
    soup = bs(response.text, 'html.parser')
    vacancies = soup.find_all('div', {'class' : 'vacancy-serp-item-body__main-info'})

    for v in vacancies:
        vac_data = {}

        title = v.find('a', {'class' : 'serp-item__title'})
        title = title.text

        salary = v.find('span', {'class': 'bloko-header-section-2'})
        try:
            salary = salary.text
        except:
            salary = None
        if salary == None:
            curr = None
        else:
            salary = str(salary)
            salary = re.sub(r'\u202f','', salary)
            curr = salary[-1]

        if salary == None:
            min_salary = None
            max_salary = None
        else:
            salary_list = salary.split()
            if salary_list[0] == 'от':
                min_salary = float(salary_list[1])
                max_salary = None
            elif salary_list[0] == 'до':
                min_salary = None
                max_salary = float(salary_list[1])
            else:
                min_salary = float(salary_list[0])
                max_salary = float(salary_list[2])

        reference = v.find('a', {'data-qa' : 'serp-item__title'})
        reference = reference.get('href')
        source = url_source

        vac_data['title'] = title
        vac_data['salary'] = salary
        vac_data['min_salary'] = min_salary
        vac_data['max_salary'] = max_salary
        vac_data['curr'] = curr
        vac_data['reference'] = reference
        vac_data['source'] = source
        
        return vac_data

In [42]:
for page in range (40):
    get_information(url, page, headers)

In [43]:
vac_dataset = pd.DataFrame(vac_data)

In [44]:
vac_dataset

,title,salary,min_salary,max_salary,curr,reference,source
0,Программист системы сбора данных Python VBA,150000 – 200000 ₽,150000.0,200000.0,₽,https://hh.ru/vacancy/86292820?query=Python+de...,https://hh.ru/vacancies/
1,Python-разработчик,до 420000 ₽,NaN,420000.0,₽,https://hh.ru/vacancy/86085894?query=Python+de...,https://hh.ru/vacancies/
2,Python разработчик,200000 – 320000 ₽,200000.0,320000.0,₽,https://hh.ru/vacancy/86285552?query=Python+de...,https://hh.ru/vacancies/
3,Python-разработчик,120000 – 150000 ₽,120000.0,150000.0,₽,https://hh.ru/vacancy/85155504?query=Python+de...,https://hh.ru/vacancies/
4,Python-разработчик,до 450000 ₽,NaN,450000.0,₽,https://hh.ru/vacancy/86120529?query=Python+de...,https://hh.ru/vacancies/
...,...,...,...,...,...,...,...
1995,Казначей,100000 – 100000 ₽,100000.0,100000.0,₽,https://hh.ru/vacancy/86285060?query=Python+de...,https://hh.ru/vacancies/
1996,Solution Архитектор,None,NaN,NaN,None,https://hh.ru/vacancy/85931388?query=Python+de...,https://hh.ru/vacancies/
1997,"Senior Data Analyst ""Growth""",None,NaN,NaN,None,https://hh.ru/vacancy/85729705?query=Python+de...,https://hh.ru/vacancies/
1998,Verification Engineer / Инженер верификатор,None,NaN,NaN,None,https://hh.ru/vacancy/85878742?query=Python+de...,https://hh.ru/vacancies/


In [51]:
vac_dataset_1 = vac_dataset[0:1850] #основная часть вакансий

In [53]:
vac_dataset_2 = vac_dataset[0:1900] #дополненный датасет вакансий

In [83]:
vac_dataset_3 = vac_dataset #еще раз дополненный датасет, моделируем ситуацию сбора информации через интервалы времени

In [45]:
import pymongo
from bson import ObjectId
from pymongo import MongoClient

In [57]:
from pprint import pprint

In [47]:
client = MongoClient('127.0.0.1', 27017)

In [48]:
db = client["vacancies"]

In [49]:
vac_hh = db.vac_hh

In [55]:
vac_hh.insert_many(vac_dataset_1.to_dict('records'))

In [59]:
vac_hh.find_one({'title': 'Python-разработчик'})

{'_id': ObjectId('64f90608a0239c17070e3734'),
 'title': 'Python-разработчик',
 'salary': 'до 420000 ₽',
 'min_salary': nan,
 'max_salary': 420000.0,
 'curr': '₽',
 'reference': 'https://hh.ru/vacancy/86085894?query=Python+developer',
 'source': 'https://hh.ru/vacancies/'}

In [73]:
vac_2 = vac_dataset_2.to_dict('records')

In [74]:
for elem in vac_2:
    ref = vac_hh.find_one({'reference': elem.get('reference')})
    if ref == None:
        vac_hh.insert_one(elem)

In [81]:
count = vac_hh.count_documents({})
print(count)

1900


In [84]:
vac_3 = vac_dataset_3.to_dict('records')

In [85]:
for elem in vac_3:
    ref = vac_hh.find_one({'reference': elem.get('reference')})
    if ref == None:
        vac_hh.insert_one(elem)

In [86]:
count = vac_hh.count_documents({})
print(count)

2000


In [87]:
param = 100000

In [89]:
salary_param_list = []
for doc in vac_hh.find({'min_salary': {'$gt': param}} or {'max_salary': {'$gt': param}}):
    salary_param_list.append(doc)
salary_param_set = pd.DataFrame(salary_param_list)

In [90]:
salary_param_set

,_id,title,salary,min_salary,max_salary,curr,reference,source
0,64f90608a0239c17070e3733,Программист системы сбора данных Python VBA,150000 – 200000 ₽,150000.0,200000.0,₽,https://hh.ru/vacancy/86292820?query=Python+de...,https://hh.ru/vacancies/
1,64f90608a0239c17070e3735,Python разработчик,200000 – 320000 ₽,200000.0,320000.0,₽,https://hh.ru/vacancy/86285552?query=Python+de...,https://hh.ru/vacancies/
2,64f90608a0239c17070e3736,Python-разработчик,120000 – 150000 ₽,120000.0,150000.0,₽,https://hh.ru/vacancy/85155504?query=Python+de...,https://hh.ru/vacancies/
3,64f90608a0239c17070e3739,Python разработчик,170000 – 265000 ₽,170000.0,265000.0,₽,https://hh.ru/vacancy/85989567?query=Python+de...,https://hh.ru/vacancies/
4,64f90608a0239c17070e373b,Middle Python-разработчик,180000 – 230000 ₽,180000.0,230000.0,₽,https://hh.ru/vacancy/86189756?query=Python+de...,https://hh.ru/vacancies/
...,...,...,...,...,...,...,...,...
239,64f91193a0239c17070e3e9c,Системный аналитик в аккредитованную компанию,от 200000 ₽,200000.0,NaN,₽,https://hh.ru/vacancy/86073778?query=Python+de...,https://hh.ru/vacancies/
240,64f91350a0239c17070e3ea0,Начальник ИТ отдела / Ведущий системный админи...,150000 – 150000 ₽,150000.0,150000.0,₽,https://hh.ru/vacancy/85122796?query=Python+de...,https://hh.ru/vacancies/
241,64f91350a0239c17070e3eaa,Начальник ИТ-отдела (специального программного...,от 160000 ₽,160000.0,NaN,₽,https://hh.ru/vacancy/81345330?query=Python+de...,https://hh.ru/vacancies/
242,64f91350a0239c17070e3eae,Программист,от 150000 ₽,150000.0,NaN,₽,https://hh.ru/vacancy/85968189?query=Python+de...,https://hh.ru/vacancies/
